# Econometrics for the Biological Sciences

## Fundamental Concepts

- Level I, II, and III analysis
- Regression
  - Study of conditional distributions
- Models vs. Algorithms
- Estimation vs. Inference
- Econometrics vs. Statistical Learning vs. ML

- Experimental vs. Quasi-Experimental vs. Observational
- Counterfactual
- Average Treatment Effect
- Average Treatment on the Treated
- Bias

### Objectives

- Identification
- Unbiased estimate of the treatment effect
- Valid standard errors for the treatment effect
- Explanation

### General Assumptions

- Make a credible case for a causal relationship, i.e. there is a model in play
- Gauss-Markov Theorem
- Neyman-Rubin Causal Model

### Some Problems

**Experimental Flaws**
- Spill over effects
- Insufficient Randomization
- **Attrition**
- **Partial Compliance**

Existence of experimental flaws requires that additional methods be applied on top of traditional methods. Usually thought of as techniques for handling quasi-experimental data

**Data Analysis/Modelling Problems**
- Model misspecification error
- Omitted variable bias (Confounding)
- Selection bias
  - Population studied may not be representative of the population of interest (returns to college education)
  - Not the same joint probability distribution
- Endogeneity Bias (Simultenaity)
  - Non-random treatment assignment
- Measurement Error
- Data Snooping/Model Selection

## Core Techniques

- Multiple Regression
- Difference-in-Difference
  - Estimate a treatment effect when baseline data is available prior to treatment
- Fixed Effects
- Matching
  - Overcome non-random treatment assignment
- Instrumental Variables
  - Overcome omitted variable bias and endogeneity bias
- Regression Discontinuity
  - Exploits thresholds (and entities just on either sides of them)
- Synthetic Controls
  - Use weighting to get a better control group

## Machine Learning for Causal Inference

- BART
- SuperLearner
- Causal Trees/Forests
- DoubleML

## Case Study: Estimating Heterogeneous Treatment Effects

## Resources

### Books
- [Statistical Learning from a Regression Perspective]
- [Data Analysis Using Regression and Multilevel/Hierarchical Models]
- [Mostly Harmless Econometrics]
- [Introduction to Statistical Learning]
- [Elements of Statistical Learning]
- [Introductory Econometrics: A Modern Approach]
- [Econometric Analysis of Cross Section and Panel Data]
- [Introduction to Econometrics]

### Papers

### Blogs
- [Causal Forest Intro](https://www.markhw.com/blog/causalforestintro)